In [38]:
from sklearn.ensemble import GradientBoostingClassifier
import conifer
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [36]:
import ipaddress

In [37]:
data = pd.read_csv("NF-ToN-IoT.csv")
data

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,192.168.1.195,63318,52.139.250.253,443,6,91.00,181,165,2,1,24,327,0,Benign
1,192.168.1.79,57442,192.168.1.255,15600,17,0.00,63,0,1,0,0,0,0,Benign
2,192.168.1.79,57452,239.255.255.250,15600,17,0.00,63,0,1,0,0,0,0,Benign
3,192.168.1.193,138,192.168.1.255,138,17,10.16,472,0,2,0,0,0,0,Benign
4,192.168.1.79,51989,192.168.1.255,15600,17,0.00,63,0,1,0,0,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379269,192.168.1.31,58032,192.168.1.194,80,6,7.00,216,180,4,3,18,9433,1,ddos
1379270,192.168.1.31,58034,192.168.1.194,80,6,7.00,216,180,4,3,18,9221,1,ddos
1379271,192.168.1.31,58036,192.168.1.194,80,6,7.00,216,180,4,3,18,9656,1,ddos
1379272,192.168.1.31,58038,192.168.1.194,80,6,7.00,216,180,4,3,18,10046,1,ddos


In [11]:
import pandas as pd
data = pd.read_csv("NF-ToN-IoT.csv")
print(data.columns)

data["IPV4_SRC_ADDR"] = data["IPV4_SRC_ADDR"].apply(lambda x: int(ipaddress.IPv4Address(x)))
data["IPV4_DST_ADDR"] = data["IPV4_DST_ADDR"].apply(lambda x: int(ipaddress.IPv4Address(x)))
data = data.drop(["Attack"], axis=1)
for col in data.columns:
    if col != "Label":
        data[col] = data[col].astype(float)
    else:
        data[col] = data[col].astype("category")
        
data["IPV4_SRC_ADDR"] = data["IPV4_SRC_ADDR"].astype("int")
data["IPV4_DST_ADDR"] = data["IPV4_DST_ADDR"].astype("int")
print(data["IPV4_SRC_ADDR"].dtype)
print(data.Label.dtype)

Index(['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
       'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'Label', 'Attack'],
      dtype='object')
int64
category


In [12]:
print(data["IPV4_SRC_ADDR"].dtype)

int64


In [19]:
# X_train=[[1,2],[3,4]]
# y_train = [3,7]
# X=numpy.array([[1,2]])

train, test = train_test_split(data, test_size=0.2, random_state=123)

X_train = train[train.columns[:-1]]
y_train = train[train.columns[-1]].astype("category")

X_test = test[test.columns[:-1]]
y_test = test[test.columns[-1]].astype("category")

In [23]:
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.inf)

np_x_test = X_test.to_numpy()
np_y_test = y_test.to_numpy()
with open("X_test.txt", "w+") as f:
    f.write(str(repr(np_x_test)).replace("]", "}").replace("[", "{"))
with open("y_test.txt", "w+") as f:
    f.write(str(repr(np_y_test)).replace("]", "}").replace("[", "{"))

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
# Train a BDT
clf = GradientBoostingClassifier(n_estimators=200).fit(X_train, y_train)

# Create a conifer config dictionary
cfg = conifer.backends.xilinxhls.auto_config()
# Change the bit precision (print the config to see everything modifiable)
cfg['Precision'] = 'ap_fixed<12,4>' 

# Convert the sklearn model to a conifer model
model = conifer.converters.convert_from_sklearn(clf, cfg)
# Write the HLS project and compile the C++-Python bridge                      
model.compile()

# Run bit-accurate prediction on the CPU

In [ ]:
import tensorflow_decision_forests as tfdf
# Train a BDT
tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="Label")

clf = tfdf.keras.GradientBoostedTreesModel(num_threads=128)
clf.fit(tf_dataset)

In [43]:
# Create a conifer config dictionary
cfg = conifer.backends.xilinxhls.auto_config()
# Change the bit precision (print the config to see everything modifiable)
cfg['Precision'] = 'ap_fixed<12,4>' 
cfg['OutputDir'] = 'tf_GBT'

# Convert the sklearn model to a conifer model
model = conifer.converters.convert_from_tf_df(clf, cfg)
# Write the HLS project and compile the C++-Python bridge                      
model.compile()

# Run bit-accurate prediction on the CPU

NameError: name 'tf_df' is not defined

In [25]:
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9998259955411357


In [11]:
y_pred

array([0, 0, 0, ..., 1, 1, 1])

In [12]:
X = X_train.head(10).astype(float).to_numpy()

In [16]:
y_hls = model.decision_function(X)
y_skl = clf.decision_function(X)

# Synthesize the model for the target FPGA
# model.build()

/home/srikarv2/miniconda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


array([-3.75193351, -5.52476574, -5.52476574, -6.2532416 , -5.52476574,
       -4.67308892, -5.52476574, -4.03133218, -5.52476574, -3.59225516])

#### !echo $XILINX_HLS

In [7]:
!echo $PATH

/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0/:/mnt/c/Windows/System32/OpenSSH/:/mnt/c/Program Files/dotnet/:/mnt/c/Program Files/PuTTY/:/mnt/c/Users/huang/AppData/Local/Microsoft/WindowsApps:/mnt/c/Users/huang/AppData/Local/Programs/Microsoft VS Code/bin:/snap/bin:/home/wendyh/.local/bin:/tools/Xilinx/Vitis_HLS/2022.1/bin/:/home/wendyh/.local/bin:/tools/Xilinx/Vitis_HLS/2022.1/bin/


In [2]:
!echo $XILINX_AP_INCLUDE

/tools/Xilinx/Vitis_HLS/2022.1/include/ap_fixed.h


In [16]:
from sklearn.ensemble import GradientBoostingClassifier
# import conifer
import pandas as pd
import numpy as np
import ipaddress

np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.inf)

# Read data 
data = pd.read_csv("ids_dataset/ids_dataset/data/NF-ToN-IoT.csv")

data["IPV4_SRC_ADDR"] = data["IPV4_SRC_ADDR"].apply(lambda x: int(ipaddress.IPv4Address(x)))
data["IPV4_DST_ADDR"] = data["IPV4_DST_ADDR"].apply(lambda x: int(ipaddress.IPv4Address(x)))
data = data.drop(["Attack"], axis=1)
for col in data.columns:
    if col == "Label":
        data[col] = data[col].astype("category")
    if col == "IPV4_SRC_ADDR" or col == "IPV4_DST_ADDR":
        data[col] = data[col].astype(int)
    else:
        data[col] = data[col].astype(float)

X_train = data.drop(["Label"], axis=1)
y_train = data["Label"]
print(X_train)

np_x_train = X_train.to_numpy()
np_y_train = y_train.to_numpy()

fmt = '%f'
file_path = 'output.txt'
np.savetxt(file_path, arr, fmt=fmt)

with open("/mnt/c/Users/huang/Downloads/X_train.txt", "w+") as f:
    f.write(str(repr(np_x_train[0:500])).replace("]", "}").replace("[", "{"))

         IPV4_SRC_ADDR  L4_SRC_PORT  IPV4_DST_ADDR  L4_DST_PORT  PROTOCOL  \
0           3232235971      63318.0      881588989        443.0       6.0   
1           3232235855      57442.0     3232236031      15600.0      17.0   
2           3232235855      57452.0     4026531834      15600.0      17.0   
3           3232235969        138.0     3232236031        138.0      17.0   
4           3232235855      51989.0     3232236031      15600.0      17.0   
...                ...          ...            ...          ...       ...   
1379269     3232235807      58032.0     3232235970         80.0       6.0   
1379270     3232235807      58034.0     3232235970         80.0       6.0   
1379271     3232235807      58036.0     3232235970         80.0       6.0   
1379272     3232235807      58038.0     3232235970         80.0       6.0   
1379273     3232235807      58040.0     3232235970         80.0       6.0   

         L7_PROTO  IN_BYTES  OUT_BYTES  IN_PKTS  OUT_PKTS  TCP_FLAGS  \
0  

In [17]:
fmt = '%f'
file_path = "/mnt/c/Users/huang/Downloads/X_train.txt"
np.savetxt(file_path, np_x_train[:10], fmt=fmt)

In [ ]:
with open("/mnt/c/Users/huang/Downloads/X_train.txt", "w+") as f:
    f.write(str(repr(np_x_train[0:500])).replace("]", "}").replace("[", "{"))

In [18]:
str(repr(np_x_train[0:500]))

'array([[3.23223597e+09, 6.33180000e+04, 8.81588989e+08, 4.43000000e+02,\n        6.00000000e+00, 9.10000000e+01, 1.81000000e+02, 1.65000000e+02,\n        2.00000000e+00, 1.00000000e+00, 2.40000000e+01, 3.27000000e+02],\n       [3.23223586e+09, 5.74420000e+04, 3.23223603e+09, 1.56000000e+04,\n        1.70000000e+01, 0.00000000e+00, 6.30000000e+01, 0.00000000e+00,\n        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n       [3.23223586e+09, 5.74520000e+04, 4.02653183e+09, 1.56000000e+04,\n        1.70000000e+01, 0.00000000e+00, 6.30000000e+01, 0.00000000e+00,\n        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n       [3.23223597e+09, 1.38000000e+02, 3.23223603e+09, 1.38000000e+02,\n        1.70000000e+01, 1.01600000e+01, 4.72000000e+02, 0.00000000e+00,\n        2.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n       [3.23223586e+09, 5.19890000e+04, 3.23223603e+09, 1.56000000e+04,\n        1.70000000e+01, 0.00000000e+00, 6.3000

In [25]:
with open("/mnt/c/Users/huang/Downloads/X_train.txt", "w+") as f:
    for element in np_x_train[0:500]:
        formatted_element = str(element)
        print(formatted_element)
        f.write(formatted_element + '\n')

[3.23223597e+09 6.33180000e+04 8.81588989e+08 4.43000000e+02
 6.00000000e+00 9.10000000e+01 1.81000000e+02 1.65000000e+02
 2.00000000e+00 1.00000000e+00 2.40000000e+01 3.27000000e+02]
[3.23223586e+09 5.74420000e+04 3.23223603e+09 1.56000000e+04
 1.70000000e+01 0.00000000e+00 6.30000000e+01 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[3.23223586e+09 5.74520000e+04 4.02653183e+09 1.56000000e+04
 1.70000000e+01 0.00000000e+00 6.30000000e+01 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[3.23223597e+09 1.38000000e+02 3.23223603e+09 1.38000000e+02
 1.70000000e+01 1.01600000e+01 4.72000000e+02 0.00000000e+00
 2.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[3.23223586e+09 5.19890000e+04 3.23223603e+09 1.56000000e+04
 1.70000000e+01 0.00000000e+00 6.30000000e+01 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[3.23223578e+09 5.39270000e+04 4.02653183e+09 1.90000000e+03
 1.70000000e+01 1.5

In [14]:
np.log(6.684140)

1.8997375560553844

In [43]:
X = X_train.astype(float).to_numpy()

In [32]:
clf.decision_function(X)

/home/wendyh/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


array([-3.75193351, -5.52476574, -5.52476574, -6.2532416 , -5.52476574,
       -4.67308892, -5.52476574, -4.03133218, -5.52476574, -3.59225516,
       -3.59225516,  6.68414109,  6.68414109,  6.68414109,  6.68414109,
        6.68414109,  6.68414109,  6.68414109,  6.68414109,  6.68414109,
        6.68414109,  6.68414109,  6.68414109,  6.68414109,  6.68414109,
        6.68414109,  6.68414109,  6.68414109,  6.68414109,  6.68414109])

In [24]:
clf.predict(X)

/home/wendyh/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [44]:
results = []
for i in range(X.shape[0]):
    result = 0 if model.decision_function(X[i]) > 0 else 1
    results.append(result)

In [45]:
label = clf.predict(X)

/home/wendyh/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [49]:
results = np.array(results)

In [50]:
np.sum(results==label)

701938

In [54]:
np.sum(results==label) / results.shape[0]

0.508918460001421